## 1) Instalar dependências (Colab)
As dependências são pesadas; use GPU em Colab. A célula abaixo instala `diffusers`, `transformers`, `accelerate`, `torch` (Colab já provê uma versão apropriada), `opencv`, `open3d`, `trimesh`, `tqdm` e utilitários.

In [1]:
# Execute esta célula no Colab (pode levar alguns minutos)
import sys
# Instala pacotes principais (não força versão de torch — Colab já possui torch com CUDA)
!pip install -q diffusers transformers accelerate safetensors==0.3.1 ftfy regex
!pip install -q opencv-python-headless pillow open3d trimesh scikit-image tqdm

  error: subprocess-exited-with-error
  
  × Building wheel for safetensors (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [36 lines of output]
      /tmp/pip-build-env-xq78gcn7/overlay/lib/python3.12/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: Apache Software License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      running build
      running build_py
      creating build/lib.linux-x86_64-cpython-312

## 2) Imports e configurações
Defina abaixo seu `HUGGINGFACE_TOKEN` (recomendado) para carregar modelos privados caso necessário.

In [2]:
from pathlib import Path
import os
import torch
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import trimesh
import open3d as o3d
from tqdm import tqdm

# Pasta de saída
OUT_DIR = Path('/content/neuroforge_outputs')
OUT_DIR.mkdir(exist_ok=True)

# Forneça seu token aqui (opcional)
HUGGINGFACE_TOKEN = os.environ.get('HF_TOKEN', '')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

OSError: libGL.so.1: cannot open shared object file: No such file or directory

## 3) Gerar imagem a partir de texto (Stable Diffusion via diffusers)
A célula abaixo usa `diffusers` para gerar uma imagem. Em Colab + GPU, use `torch_dtype=torch.float16` para acelerar.

In [ ]:
from diffusers import StableDiffusionPipeline

def generate_image(prompt: str, num_inference_steps: int = 20, height: int = 512, width: int = 512, seed: int = None):
    if seed is None:
        generator = None
    else:
        generator = torch.Generator(device).manual_seed(seed)

    model_id = 'runwayml/stable-diffusion-v1-5'
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16 if device=='cuda' else torch.float32)
    pipe = pipe.to(device)

    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, generator=generator).images[0]
    return image

# Exemplo de uso (substitua o prompt):
prompt = 'A detailed 3D printable model of a futuristic vase, white porcelain, studio lighting'
img = generate_image(prompt, num_inference_steps=25, height=512, width=512, seed=42)
img.save(OUT_DIR / 'generated_image.png')
display(img)

## 4) Estimar mapa de profundidade (MiDaS via torch.hub)
Usamos o repositório `intel-isl/MiDaS` via `torch.hub`. A estimativa produz um mapa de profundidade relativo que será normalizado.

In [ ]:
# Carregar modelo MiDaS
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS')
midas_transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = midas_transforms.default_transform
midas.to(device).eval()

# Carregar a imagem gerada
img = Image.open(OUT_DIR / 'generated_image.png').convert('RGB')
img_cv = np.array(img)[:, :, ::-1]  # RGB->BGR for OpenCV if needed
input_batch = transform(img).to(device)
with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1), size=img.size[::-1], mode='bicubic', align_corners=False
    ).squeeze()
depth_map = prediction.cpu().numpy()
# Normalizar para visualização
depth_norm = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())
plt.figure(figsize=(8,8)); plt.axis('off'); plt.imshow(depth_norm, cmap='viridis'); plt.title('Estimated depth'); plt.show()
Image.fromarray((depth_norm*255).astype('uint8')).save(OUT_DIR / 'depth_map.png')

## 5) Converter Depth → Point Cloud
Usamos uma câmera pinhole simples para projetar pontos 3D a partir do mapa de profundidade. Ajuste `focal_length` conforme necessário.

In [ ]:
def depth_to_pointcloud(depth, img_rgb, focal_length=1.0, scale=1.0):
    h, w = depth.shape
    i, j = np.meshgrid(np.arange(w), np.arange(h), indexing='xy')
    # coordenadas normalizadas (centered)
    cx = w / 2.0
    cy = h / 2.0
    fx = focal_length * w
    fy = focal_length * h
    z = depth * scale
    x = (i - cx) * z / fx
    y = (j - cy) * z / fy
    points = np.stack((x, -y, z), axis=-1).reshape(-1, 3)
    colors = np.array(img_rgb).reshape(-1, 3) / 255.0
    # Filtrar pontos muito distantes/NaN
    mask = np.isfinite(points).all(axis=1) & (z.reshape(-1) > 0)
    points = points[mask]
    colors = colors[mask]
    return points, colors

img = Image.open(OUT_DIR / 'generated_image.png').convert('RGB').resize(depth_norm.shape[::-1])
points, colors = depth_to_pointcloud(depth_norm, img, focal_length=1.2, scale=1.0)
len(points)

## 6) Reconstruir malha (Poisson) e salvar como STL
Usamos `open3d` para estimar normais e executar reconstrução Poisson. Ajuste `depth` (Poisson) conforme necessário — valores maiores geram mais detalhes, porém custam mais memória.

In [ ]:
# Criar PointCloud Open3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)
# Estimar normais
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))
pcd.orient_normals_consistent_tangent_plane(30)

# Filtrar pontos isolados (opcional)
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd = pcd.select_by_index(ind)

# Reconstrução Poisson
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8)

# Optional: crop by density to remove low-density artifacts
dens = np.asarray(densities)
density_threshold = np.quantile(dens, 0.01)
vertices_to_keep = dens > density_threshold
# Note: simple cropping via density can be done by removing vertices — here we skip complex cropping for brevity

# Smooth and simplify (optional)
mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=200000)
mesh.compute_vertex_normals()

# Salvar como STL
out_path = OUT_DIR / 'output_mesh.stl'
o3d.io.write_triangle_mesh(str(out_path), mesh)
print('Saved mesh to', out_path)

## 7) Visualizar e baixar o arquivo
A célula abaixo mostra a malha e fornece link para download no ambiente Colab.

In [ ]:
# Visualizar com trimesh (render simples)
import trimesh
mesh_tr = trimesh.load_mesh(str(out_path))
display(mesh_tr)

In [ ]:
# Fornece link para download no Colab
from google.colab import files
files.download(str(out_path))

---
Observações finais: esta pipeline é de prototipagem e produz uma malha aproximada. Ajustes necessários: calibração de `focal_length`, pós-processamento da malha (fechamento, decimação, remediação), uso de modelos condicionados para melhorar detalhes e escalonamento para impressão 3D.